# 1. Mounting the Dataset to Colab's Storage

In [ ]:
# Ensure a stable version of the Kaggle API client library is installed
# This is required to download datasets from Kaggle
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=fafcbd66cb2a94dd1ff091f78ce09cd19eb715c6d0b3924657de5d1690ddb35f
  Stored in directory: /root/.cache/pip/wheels/f5/69/4d/d701fc604b9fb09be59718b4056fd5556a22588ce1f25dd090
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.5
    Uninstalling kaggle-1.7.4.5:
      Successfully uninstalled kaggle-1.7.4.5


In [ ]:
import os
from google.colab import userdata

# Set the variable equal to your Kaggle username and API key
# These credentials are required to authenticate with Kaggle API
# You can get your API key from: https://www.kaggle.com/settings -> API section
os.environ["KAGGLE_USERNAME"] =  # Add your Kaggle username here
os.environ["KAGGLE_KEY"] =  # Add your Kaggle API key here
print("Kaggle API credentials configured as environment variables successfully.")

SyntaxError: invalid syntax (ipython-input-2976768983.py, line 4)

In [ ]:
# Download the TMDB movie metadata dataset from Kaggle
# This dataset contains information about 5000 movies including credits and metadata
!kaggle datasets download tmdb/tmdb-movie-metadata

 56% 5.00M/8.89M [00:00<00:00, 20.6MB/s]
100% 8.89M/8.89M [00:00<00:00, 34.6MB/s]


In [ ]:
# Extract the downloaded zip file
# This will create two CSV files: tmdb_5000_movies.csv and tmdb_5000_credits.csv
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [ ]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import ast           # For safely evaluating strings containing Python literals

# 2. Loading and Exploring the Dataset

In [ ]:
# Load the datasets
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv('tmdb_5000_credits.csv')

# Display first few rows to understand the data structure
print("Movies dataset:")
movies.head()
print("\nCredits dataset:")
credits.head()

# Merge the two datasets on the 'title' column
# This combines movie metadata with cast and crew information
movies = movies.merge(credits, on='title')

# Display merged dataset and column names
print("\nMerged dataset:")
movies.head()
print("\nColumn names:")
movies.columns

In [ ]:
# Select only the relevant columns for our recommendation system
# We need: movie_id, title, genres, keywords, overview, cast, and crew
movies = movies[['movie_id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]


In [ ]:
movies.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


### 2.1. Demo: Converting Attributes to Usable Format

This section demonstrates conversion functions for extracting key information from complex attributes. 
Here we use the 'genres' attribute and extract the 'name' field from each genre dictionary.


In [ ]:
# Demo conversion function before creating a general function
# This function converts string representations of lists/dicts into actual Python lists
# and extracts the 'name' field from dictionary items
def convert(obj):
    """
    Convert string representations of lists/dictionaries to Python lists.
    Extracts 'name' field from dictionary items.
    
    Args:
        obj: Can be a list, string, or other type
        
    Returns:
        List of extracted names or empty list if conversion fails
    """
    if isinstance(obj, list):
        return obj
    if isinstance(obj, str):
        try:
            # Safely evaluate string as Python literal (list/dict)
            parsed_data = ast.literal_eval(obj)
            L = []
            for item in parsed_data:
                # Extract 'name' from dictionary items
                if isinstance(item, dict) and 'name' in item:
                    L.append(item['name'])
                elif isinstance(item, str):
                    L.append(item)
            return L
        except (ValueError, SyntaxError):
            return []
    return []

# Apply conversion to genres column
# Note: This is a demo - we'll use a more robust function later
movies['genres'] = movies['genres'].apply(convert)

/tmp/ipython-input-128248253.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = movies['genres'].apply(convert)


### 2.2. Testing the Demo Function

In [ ]:
# Test the convert function on keywords column
movies['keywords'] = movies['keywords'].apply(convert)

/tmp/ipython-input-3328512696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
movies['keywords'].head()

,keywords
0,"[culture clash, future, space war, space colon..."
1,"[ocean, drug abuse, exotic island, east india ..."
2,"[spy, based on novel, secret agent, sequel, mi..."
3,"[dc comics, crime fighter, terrorist, secret i..."
4,"[based on novel, mars, medallion, space travel..."


In [ ]:
# Test the convert function on keywords column (duplicate - can be removed)
movies['keywords'] = movies['keywords'].apply(convert)

/tmp/ipython-input-3328512696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
movies['keywords'].head()

,keywords
0,"[culture clash, future, space war, space colon..."
1,"[ocean, drug abuse, exotic island, east india ..."
2,"[spy, based on novel, secret agent, sequel, mi..."
3,"[dc comics, crime fighter, terrorist, secret i..."
4,"[based on novel, mars, medallion, space travel..."


In [ ]:
movies['genres']

,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4804,"[Action, Crime, Thriller]"
4805,"[Comedy, Romance]"
4806,"[Comedy, Drama, Romance, TV Movie]"
4807,[]


# 3. Converting Attributes to Usable Format

This section implements a unified function to process all metadata attributes (genres, keywords, cast, crew) 
by extracting key information from them in a consistent manner.

In [ ]:
# Unified function to process metadata lists
# This function handles conversion of string representations to lists and extracts relevant information
def process_metadata_list(obj, max_items=None, filter_key=None, filter_value=None):
    """
    Process metadata lists by converting string representations to Python lists
    and extracting key information (like 'name' field from dictionaries).
    
    Args:
        obj: String representation of a list/dict or an actual list
        max_items: Maximum number of items to extract (None for all)
        filter_key: Key to filter on (e.g., 'job' for crew members)
        filter_value: Value to match for filter_key (e.g., 'Director')
        
    Returns:
        List of extracted names/values
    """
    # Ensure obj is a list, converting from string if necessary
    if isinstance(obj, str):
        try:
            # Safely evaluate string as Python literal
            parsed_list = ast.literal_eval(obj)
        except (ValueError, SyntaxError):
            return []
    elif isinstance(obj, list):
        parsed_list = obj
    else:
        return []

    result_names = []
    item_count = 0
    
    for item in parsed_list:
        include_item = False
        
        if filter_key:
            # Apply filter if specified (e.g., only get Directors from crew)
            if isinstance(item, dict) and item.get(filter_key) == filter_value:
                include_item = True
        else:
            # No filter, include if it has a 'name' or is already a string
            if isinstance(item, dict) and 'name' in item:
                include_item = True
            elif isinstance(item, str):
                include_item = True

        if include_item:
            # Extract the name from dictionary or use the string directly
            name = item.get('name') if isinstance(item, dict) else item
            if name is not None:  # Ensure name is not None
                result_names.append(name)
                item_count += 1

        # Stop if we've reached the maximum number of items
        if max_items is not None and item_count >= max_items:
            break

    return result_names


# 4. Load Data Function

This function loads and preprocesses the movie data, combining metadata from both datasets
and preparing it for the recommendation system.

In [ ]:
def load_data():
    """
    Load and preprocess movie data for the recommendation system.
    
    This function:
    1. Loads movies and credits datasets
    2. Merges them on title
    3. Processes metadata columns (genres, keywords, cast, crew)
    4. Creates a combined 'tags' column for content-based filtering
    
    Returns:
        DataFrame with processed movie data containing movie_id, title, and tags
    """
    # Reload the initial dataframes to ensure a clean state
    movies = pd.read_csv('tmdb_5000_movies.csv')
    credits = pd.read_csv('tmdb_5000_credits.csv')

    # Merge the dataframes on the 'title' column
    # This combines movie metadata with cast and crew information
    movies = movies.merge(credits, on='title')

    # Select only the relevant columns for our recommendation system
    movies = movies[['movie_id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

    # Apply the unified function to each column with appropriate parameters
    # Genres: Extract all genre names
    movies['genres'] = movies['genres'].apply(process_metadata_list)
    
    # Keywords: Extract all keyword names
    movies['keywords'] = movies['keywords'].apply(process_metadata_list)
    
    # Cast: Extract top 3 cast members (most prominent actors)
    movies['cast'] = movies['cast'].apply(process_metadata_list, max_items=3)
    
    # Crew: Extract only the Director (filter by job='Director', max 1)
    movies['crew'] = movies['crew'].apply(process_metadata_list, filter_key='job', filter_value='Director', max_items=1)

    # Process overview column: split into words for better text processing
    movies['overview'] = movies['overview'].fillna('')  # Handle missing values
    movies['overview'] = movies['overview'].apply(lambda x: x.split())

    # Create the tags column by combining all relevant information
    # This creates a comprehensive text representation of each movie
    movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
    
    # Convert tags list to lowercase string for TF-IDF processing
    movies['tags'] = movies['tags'].apply(lambda x: ' '.join(x).lower())
    
    # Drop the individual columns as they're now combined in 'tags'
    movies = movies.drop(columns=['overview', 'genres', 'keywords', 'cast', 'crew'])
    
    return movies

# 5. Loading Data and Building the Recommendation Model

This section loads the processed data and builds the recommendation system using TF-IDF vectorization
and cosine similarity.

In [ ]:
# Load and preprocess the movie data
df = load_data()

In [ ]:
df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [ ]:
# Import required libraries for text vectorization and similarity calculation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize TF-IDF Vectorizer
# TF-IDF (Term Frequency-Inverse Document Frequency) converts text into numerical vectors
# stop_words='english' removes common English words that don't add meaning
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the tags column to create a TF-IDF matrix
# Each movie is represented as a vector in high-dimensional space
# Rows = movies, Columns = unique words/terms
tfidf_matrix = tfidf.fit_transform(df['tags'])

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Number of movies: {tfidf_matrix.shape[0]}")
print(f"Number of unique terms: {tfidf_matrix.shape[1]}")



In [ ]:
# Calculate cosine similarity matrix
# Cosine similarity measures the cosine of the angle between two vectors
# Values range from 0 (no similarity) to 1 (identical)
# This creates a matrix where each cell (i,j) represents similarity between movie i and movie j
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"Cosine Similarity Matrix Shape: {cosine_sim.shape}")
print("Matrix is symmetric - similarity between movie i and j equals similarity between j and i")

In [ ]:
def get_recommendations(title, cosine_sim_matrix=cosine_sim, df_data=df):
    """
    Get movie recommendations based on content similarity.
    
    This function finds movies similar to the input movie by:
    1. Finding the input movie in the dataset
    2. Getting similarity scores with all other movies
    3. Returning the top 10 most similar movies
    
    Args:
        title: Title of the movie to get recommendations for
        cosine_sim_matrix: Pre-computed cosine similarity matrix (default: cosine_sim)
        df_data: DataFrame containing movie data (default: df)
        
    Returns:
        Series of recommended movie titles (top 10) or error message if movie not found
    """
    # Get the index of the movie that matches the title
    # Use .index[0] to get the actual integer index of the first matching row
    try:
        idx = df_data[df_data['title'] == title].index[0]
    except IndexError:
        return f"Movie '{title}' not found in the dataset."

    # Get the pairwise similarity scores of all movies with the input movie
    # enumerate creates (index, similarity_score) pairs
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the movies based on the similarity scores (descending order)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    # Start from index 1 to exclude the movie itself (which has similarity = 1.0)
    sim_scores = sim_scores[1:11]

    # Extract the movie indices from the similarity scores
    movie_indices = [i[0] for i in sim_scores]

    # Return the titles of the top 10 most similar movies
    return df_data['title'].iloc[movie_indices]

# 6. Testing the Recommendation System

Test the recommendation system by getting movie recommendations for a sample movie.

In [ ]:
# Test the recommendation system with the movie "Memento"
# This will return 10 movies similar to Memento based on content similarity
print("Recommendations for 'Memento':")
print(get_recommendations("Memento"))

3983                                 Amnesiac
1467                          The Maze Runner
1764                                Dark City
3536                                     Iris
207                              Total Recall
3207                             The I Inside
515                            50 First Dates
2154    Eternal Sunshine of the Spotless Mind
1810                                Self/less
1034                                 Insomnia
Name: title, dtype: object


In [ ]:
df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...
